In [1]:
!pip install --pre deepchem

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 828.3/828.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 24.4 MB/s eta 0:00:00


## ***Dataset***

In [2]:
import pandas as pd
df=pd.read_csv('/content/hERG_bioactivity_pIC50.csv')
df

,assay_chembl_id,assay_description,canonical_smiles,Source,Name,hERG_uM,Activity,pIC50,hERG_Activity
0,CHEMBL841079,Inhibition of hERG currents Kv11.1,O=C1NCCN1CCN1CCC(c2cn(-c3ccc(F)cc3)c3ccc(Cl)cc...,J Med Chem,CHEMBL12713,0.0140,Yes,7.853872,1
1,CHEMBL691014,K+ channel blocking activity in human embryoni...,O=C(CCCN1CC=C(n2c(=O)[nH]c3ccccc32)CC1)c1ccc(F...,J Med Chem,CHEMBL1108,0.0322,Yes,7.492144,1
2,CHEMBL691014,K+ channel blocking activity in human embryoni...,COc1ccc(CCN(C)CCCC(C#N)(c2ccc(OC)c(OC)c2)C(C)C...,J Med Chem,CHEMBL6966,0.1430,Yes,6.844664,1
3,CHEMBL877203,K+ channel blocking activity in Chinese hamste...,CCCCN(CCCC)CCC(O)c1cc2c(Cl)cc(Cl)cc2c2cc(C(F)(...,J Med Chem,CHEMBL1107,0.1960,Yes,6.707744,1
4,CHEMBL691014,K+ channel blocking activity in human embryoni...,CCOC(=O)N1CCC(=C2c3ccc(Cl)cc3CCc3cccnc32)CC1,J Med Chem,CHEMBL998,0.1730,Yes,6.761954,1
...,...,...,...,...,...,...,...,...,...
2963,CHEMBL5048865,Inhibition of hERG by patch clamp method,CCOP(=O)(Cn1ccc(NC(=O)c2cc(Oc3ccc(S(C)(=O)=O)c...,J Med Chem,CHEMBL5081517,44.0000,No,4.356547,0
2964,CHEMBL5048865,Inhibition of hERG by patch clamp method,CCOP(=O)(Cn1ccc(NC(=O)c2cc(Oc3ccc(S(=O)(=O)N4C...,J Med Chem,CHEMBL5072442,33.0000,No,4.481486,0
2965,CHEMBL5049389,Inhibition of hERG,Cc1nc(C)c([C@H](OC(C)(C)C)C(=O)O)c(N2CCC(C)(C)...,J Med Chem,CHEMBL5093378,0.6700,Yes,6.173925,1
2966,CHEMBL5050750,Inhibition of human ERG,Cc1cnc(Nc2ccnn2C)nc1-c1cc2n(c1)C(=O)N([C@H](CO...,ACS Med Chem Lett,CHEMBL5070887,0.0140,Yes,7.853872,1


In [18]:
df = r"/content/hERG_bioactivity_pIC50.csv"
df

'/content/hERG_bioactivity_pIC50.csv'

In [5]:
import deepchem as dc

In [19]:
loader = dc.data.CSVLoader(tasks=["hERG_Activity"],
                                 smiles_field="canonical_smiles",
                                 featurizer=dc.feat.CircularFingerprint(size=2048, radius=4))

In [20]:
dataset = loader.featurize(df)

/usr/local/lib/python3.10/dist-packages/deepchem/data/data_loader.py:168: FutureWarning: featurize() is deprecated and has been renamed to create_dataset().featurize() will be removed in DeepChem 3.0
  warnings.warn(


In [21]:
   # Splitter splits the dataset
    # In this case it's is an equivalent of train_test_split from sklearn
splitter = dc.splits.ScaffoldSplitter()
    # frac_test is 0.01 because we only use a train and valid as an example
train, valid, test = splitter.train_valid_test_split(dataset,
                                                      frac_train=0.7,
                                                      frac_valid=0.29,
                                                      frac_test=0.01)
    # Normalizer will normalize y values in the dataset
normalizer = dc.trans.NormalizationTransformer(transform_y=True,
                                                         dataset=train,
                                                         move_mean=True)
train = normalizer.transform(train)
valid = normalizer.transform(valid)
test = normalizer.transform(test)

In [22]:
print(f"Size of the training data: {len(train.ids)}")
print(f"Size of the validation data: {len(valid.ids)}")
print(test)

Size of the training data: 2077
Size of the validation data: 861
<DiskDataset X.shape: (30, 2048), y.shape: (30, 1), w.shape: (30, 1), ids: ['CN(C(=O)CC[C@@H](C1CCCCC1)N1Cc2cc(Oc3ccccc3)ccc2N=C1N)C1CCCCC1'
 'N[C@H]1CN(c2ccn3cnnc3n2)CC[C@@H]1c1cc(F)c(F)cc1F.O=C(O)C(F)(F)F'
 'N[C@H]1CN(c2ccc3nc(=O)ccn3n2)CC[C@@H]1c1cc(F)c(F)cc1F.O=C(O)C(F)(F)F'
 ... 'CS(=O)(=O)Nc1ccc2c(c1)C(=O)CC1(CCN(CCc3ccc4nonc4c3)CC1)O2'
 'CN(C)CCCn1nc(C2=C(c3cn(-c4ccc5ccccc5c4)c4ccccc34)C(=O)NC2=O)c2ccccc21'
 'CN(C)CCN1C(=O)C(NC(=O)CCc2ccc(Cl)cc2Cl)N=C(c2ccccc2)c2ccccc21'], task_names: ['hERG_Activity']>


## ***Keras Model***

KerasModel is a subclass of DeepChem's Model class. It acts as a wrapper around a tensorflow.keras.Model

In [23]:
import deepchem as dc
import tensorflow as tf

In [24]:
class ClassificationModel(tf.keras.Model):

    def __init__(self):
        super(ClassificationModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(1000, activation='relu')
        self.dense2 = tf.keras.layers.Dense(1)

    def call(self, inputs, training=False):
        y = self.dense1(inputs)
        if training:
            y = tf.nn.dropout(y, 0.5)
        logits = self.dense2(y)
        output = tf.nn.sigmoid(logits)
        return output, logits

keras_model = ClassificationModel()
output_types = ['prediction', 'loss']
model = dc.models.KerasModel(keras_model, dc.models.losses.SigmoidCrossEntropy(), output_types=output_types)

In [25]:
model.fit(train, nb_epoch=100)
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)
print('training set score:', model.evaluate(train, [metric]))
print('test set score:', model.evaluate(test, [metric]))

training set score: {'roc_auc_score': 0.408207343412527}
test set score: {'roc_auc_score': 0.47058823529411764}


## ***Torch Model***

In [26]:
import torch

In [30]:
class ClassificationModel(torch.nn.Module):

    def __init__(self):
        super(ClassificationModel, self).__init__()
        self.dense1 = torch.nn.Linear(2048, 1000)
        self.dense2 = torch.nn.Linear(1000, 1)

    def forward(self, inputs):
        y = torch.nn.functional.relu( self.dense1(inputs) )
        y = torch.nn.functional.dropout(y, p=0.5, training=self.training)
        logits = self.dense2(y)
        output = torch.sigmoid(logits)
        return output, logits

torch_model = ClassificationModel()
output_types = ['prediction', 'loss']
model = dc.models.TorchModel(torch_model, dc.models.losses.SigmoidCrossEntropy(), output_types=output_types)

In [31]:
model.fit(train, nb_epoch=100)
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)
print('training set score:', model.evaluate(train, [metric]))
print('test set score:', model.evaluate(test, [metric]))

training set score: {'roc_auc_score': 0.4137868970482361}
test set score: {'roc_auc_score': 0.47058823529411764}


In [32]:
print('valid set score:', model.evaluate(valid, [metric]))

valid set score: {'roc_auc_score': 0.47134387351778656}
